## Clip old/new CHMs that overlap, difference them, and save as new .tif
mjf 10/24/2024

In [1]:
#!pip install  pyhdf

In [2]:
import pandas as pd
import numpy as np
import os
os.environ['USE_PYGEOS'] = '0'
os.umask(0o022)
import glob
import time
import datetime
import re

from shapely.geometry import Polygon, box, mapping
import rasterio as rio

import geopandas as gpd
from osgeo import gdal, gdal_array
from osgeo.osr import SpatialReference
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.plot import show
from rasterio.mask import mask
from rasterio.merge import merge
import contextily as ctx

from sklearn.linear_model import TheilSenRegressor
from scipy.stats import kendalltau

#from pyhdf.SD import SD, SDC

import matplotlib.pyplot as plt

In [3]:
srlite_dir = r'/explore/nobackup/people/mfrost2/projects/srlite/AGU_AK_2024/'
chm_dir = r'/explore/nobackup/projects/above/misc/ABoVE_Shrubs/chm_2024/'

In [4]:
ref_df = gpd.read_file(os.path.join(srlite_dir,'clipped_scenes.shp'))
ref_df

,geometry_o,catid_old,sensor_old,acq_year_o,acq_month_,cloudcover,geometry_n,catid_new,sensor_new,acq_year_n,acq_mont_1,cloudcov_1,year_diff,old_avail,new_avail,geometry
0,POLYGON ((-161.4595030393477 58.76977186288099...,1010010000B62600,QB02,2002,7,-999.000000,POLYGON ((-161.49718944245345 59.1062370667790...,10300100EA4A0000,WV02,2023,0.124500,0.124500,21,1,1,"POLYGON ((-161.43689 59.58649, -161.43970 59.4..."
1,POLYGON ((-161.4595030393477 58.76977186288099...,1010010000B62600,QB02,2002,7,-999.000000,POLYGON ((-161.51348877799998 58.9888916100000...,10300100EA7FDF00,WV02,2023,0.118800,0.118800,21,1,1,"POLYGON ((-161.43689 59.58649, -161.43970 59.4..."
2,POLYGON ((-161.4595030393477 58.76977186288099...,1010010000B62600,QB02,2002,7,-999.000000,MULTIPOLYGON (((-162.0264713782611 59.16070584...,10300100EAA7F200,WV02,2023,0.103222,0.103222,21,1,1,"MULTIPOLYGON (((-161.74821 59.04788, -161.7438..."
3,POLYGON ((-161.4595030393477 58.76977186288099...,1010010000B62600,QB02,2002,7,-999.000000,MULTIPOLYGON (((-161.76672363899996 58.9873046...,10300100EB270B00,WV02,2023,0.118444,0.118444,21,1,1,"MULTIPOLYGON (((-161.74821 59.04788, -161.7438..."
4,POLYGON ((-160.12078532233554 58.5577435845913...,1010010000C3DA00,QB02,2002,7,-999.000000,POLYGON ((-159.82849120399996 58.5653076170000...,10300100D85C8100,WV02,2022,0.005000,0.005000,20,1,1,"POLYGON ((-160.12067 58.66517, -159.82861 58.6..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
748,POLYGON ((-162.84727833999997 64.7350785900000...,1010010008831100,QB02,2008,8,0.078667,POLYGON ((-163.21388735786292 64.5095300912349...,10300100ECA8C800,WV02,2023,0.009500,0.009500,15,1,1,"POLYGON ((-162.84602 64.55658, -163.21400 64.5..."
749,POLYGON ((-162.84727833999997 64.7350785900000...,1010010008831100,QB02,2008,8,0.078667,POLYGON ((-163.42141133448993 64.9423125753080...,105001003537F700,GE01,2023,0.056714,0.056714,15,1,1,"POLYGON ((-163.22848 64.71513, -163.22767 64.8..."
750,POLYGON ((-160.6473381995104 66.94411204142365...,1010010008831200,QB02,2008,8,0.000000,POLYGON ((-160.96032963827682 66.7287011938559...,10300100C4266B00,WV02,2021,0.021400,0.021400,13,1,1,"POLYGON ((-160.64775 67.08353, -160.52292 67.0..."
751,POLYGON ((-160.6473381995104 66.94411204142365...,1010010008831200,QB02,2008,8,0.000000,POLYGON ((-160.15692138499998 66.5908813520000...,10300100C42A0900,WV02,2021,0.002600,0.002600,13,1,1,"POLYGON ((-160.22575 67.07334, -160.22558 66.9..."


In [5]:
# Get a list of files in the directory
chm_files = glob.glob(f'{chm_dir}**/*.tif', recursive = True)

chm_combined = '\t'.join(chm_files)

matching_rows = []
# Function to find matching files

for index, row in ref_df.iterrows():
    if row['catid_old'] in chm_combined and row['catid_new'] in chm_combined:
        #print(row)
        matching_rows.append(row)

# Create a new DataFrame from the matching rows
df = gpd.GeoDataFrame(matching_rows, geometry='geometry', ).reset_index(drop=True).set_crs(4326)
print(df.crs)

src1 = rio.open(chm_files[0])
print(src1.crs)
df_reproj = df.to_crs(src1.crs)

# Display the filtered DataFrame
df_reproj.head()

ValueError: Unknown column geometry

In [ ]:
for index, row in df.iterrows():
    id1 = row['catid_old']
    id2 = row['catid_new']
    file1 = glob.glob(f'{chm_dir}**/*{id1}*.tif')[0]
    file2 = glob.glob(f'{chm_dir}**/*{id2}*.tif')[0]
    # file1 = glob.glob(f'{chm_dir}**/*{row['catid_old']}*.tif')
    # if row['catid_old'] in chm_combined and row['catid_new'] in chm_combined:
    #     #print(row)
    #     matching_rows.append(row)

In [ ]:
ax = df.plot(column='year_diff', cmap='plasma', alpha=0.5, figsize=(5,10), legend=True)
ctx.add_basemap(ax=ax, crs=4326, source = ctx.providers.Esri.WorldGrayCanvas, attribution=' ')

In [ ]:
id1 = df_reproj['catid_old'][0]
id2 = df_reproj['catid_new'][0]

tif_file1 = glob.glob(f'{chm_dir}**/*{id1}*.tif')[0]
tif_file2 = glob.glob(f'{chm_dir}**/*{id2}*.tif')[0]
base1 = os.path.basename(tif_file1)
name1 = base1[5:9]+"_"+base1[19:35]
base2 = os.path.basename(tif_file2)
name2 = base2[5:9]+"_"+base2[19:35]
name_combo = name1+"-"+name2
print(name_combo)
output_file = f'{srlite_dir}clipped_diffs/{name_combo}.tif'

# src1 = rio.open(raster_file1)
# print(src1.crs)
# df_reproj = df.to_crs(src1.crs)
geometry = df_reproj['geometry'][0]

# Function to read and clip a TIFF file
def read_and_clip_tif(tif_file, geometry):
    src = rio.open(tif_file)
    # Clip the raster with the given geometry
    out_image, out_transform = mask(src, [mapping(geometry)], crop=True)
    return out_image[0], out_transform  # Return the first band as an array

# Read and clip the two TIFF files
array1, out_transform = read_and_clip_tif(tif_file1, geometry)
array1[array1 < 0] = np.nan
array2, out_transform = read_and_clip_tif(tif_file2, geometry)
array2[array2 < 0] = np.nan

# Calculate the difference
difference_array = array2 - array1
print(difference_array.min())

# Save the difference array as a new TIFF file
with rio.open(tif_file1) as src:
    meta = src.meta.copy()
    meta.update({
        'count': 1,
        'dtype': 'float32',  # Use float to handle negative values if needed
        'height': difference_array.shape[0],
        'width': difference_array.shape[1],
        'transform': out_transform,
    })

    with rio.open(output_file, 'w', **meta) as dst:
        dst.write(difference_array, 1)
        
print(f'Difference TIFF saved as {output_file}')

# Create a plot
src_diff = rio.open(output_file)
raster_diff = src_diff.read(1)  # Read the first band
transform_diff = src_diff.transform

fig, ax = plt.subplots(figsize=(12, 12))

# Plot the first raster
ax = rio.plot.show(raster_diff, ax=ax, transform=out_transform, cmap='RdYlGn')#, alpha=0.5)
# Show the plot

plt.show()


In [ ]:
# File paths for the two raster files
raster_file1 = tif_file1
raster_file2 = tif_file2
#gdf_path = 'path/to/your/geodataframe.gpkg'  # Path to the GeoDataFrame

# Read the rasters
with rio.open(raster_file1) as src1:
    print(src1.crs)
    raster1 = src1.read(1)  # Read the first band
    transform1 = src1.transform
    

with rio.open(raster_file2) as src2:
    print(src2.crs)
    raster2 = src2.read(1)  # Read the first band
    transform2 = src2.transform

# Read the geometries from the GeoDataFrame
#gdf = gpd.read_file(gdf_path)
gdf = df.head(1)
gdf = gdf.to_crs(src1.crs)
print(gdf.crs)

# Create a plot
fig, ax = plt.subplots(figsize=(12, 12))

# Plot the first raster
rio.plot.show(raster1, ax=ax, transform=transform1, cmap='Blues', alpha=0.5)

# Plot the second raster
rio.plot.show(raster2, ax=ax, transform=transform2, cmap='Reds', alpha=0.5)

# # Plot the geometries
#gdf.plot(ax=ax, color='none', edgecolor='black')

# Set plot title and labels
ax.set_title('Raster and Geometry Overlay', fontsize=15)
ax.set_xlabel('Longitude', fontsize=12)
ax.set_ylabel('Latitude', fontsize=12)

#ctx.add_basemap(ax=ax, crs=gdf.crs, source = ctx.providers.Esri.WorldGrayCanvas, attribution=' ')

# Show the plot
plt.show()


In [ ]:
# Function to read and clip a TIFF file
def read_and_clip_tif(tif_file, geometry):
    src = rio.open(tif_file)
    # Clip the raster with the given geometry
    out_image, out_transform = mask(src, [mapping(geometry)], crop=True)
    return out_image[0], out_transform  # Return the first band as an array

def clip_2tifs(id1, id2, geometry):
# id1 = df['catid_old'][0]
# id2 = df['catid_new'][0]

    tif_file1 = glob.glob(f'{chm_dir}**/*{id1}*.tif')[0]
    tif_file2 = glob.glob(f'{chm_dir}**/*{id2}*.tif')[0]
    base1 = os.path.basename(tif_file1)
    name1 = base1[5:9]+"_"+base1[19:35]
    base2 = os.path.basename(tif_file2)
    name2 = base2[5:9]+"_"+base2[19:35]
    name_combo = name1+"-"+name2
    print(name_combo)
    output_file = f'{srlite_dir}clipped_diffs/{name_combo}.tif'

    # src1 = rio.open(raster_file1)
    # print(src1.crs)
    # df_reproj = df.to_crs(src1.crs)
    # geometry = df_reproj['geometry'][0]

    # Read and clip the two TIFF files
    array1, out_transform = read_and_clip_tif(tif_file1, geometry)
    array1[array1 < 0] = np.nan
    print("Array 1 Min: ",np.nanmin(array1), "Max: ",np.nanmax(array1))
    array2, out_transform = read_and_clip_tif(tif_file2, geometry)
    array2[array2 < 0] = np.nan
    print("Array 2 Min: ",np.nanmin(array2), "Max: ",np.nanmax(array2))

    # Calculate the difference
    difference_array = array2 - array1
    print("Diff Array Min: ",np.nanmin(difference_array), "Max: ",np.nanmax(difference_array))

    # Save the difference array as a new TIFF file
    with rio.open(tif_file1) as src:
        meta = src.meta.copy()
        meta.update({
            'count': 1,
            'dtype': 'float32',  # Use float to handle negative values if needed
            'height': difference_array.shape[0],
            'width': difference_array.shape[1],
            'transform': out_transform,
        })

        with rio.open(output_file, 'w', **meta) as dst:
            dst.write(difference_array, 1)

    print(f'Difference TIFF saved as {output_file}')

    # Create a plot
    src_diff = rio.open(output_file)
    raster_diff = src_diff.read(1)  # Read the first band
    #transform_diff = src_diff.transform

    fig, ax = plt.subplots(figsize=(12, 12))

    # Plot the first raster
    ax = rio.plot.show(raster_diff, ax=ax, transform=transform1, cmap='RdYlGn')#, alpha=0.5)
    # Show the plot

    plt.show()

In [ ]:
id1 = df_reproj['catid_old'][2]
id2 = df_reproj['catid_new'][2]
geometry = df_reproj['geometry'][2]
clip_2tifs(id1, id2, geometry)

In [ ]:
#for index, row in df_reproj.tail(2).iterrows():
for index, row in df_reproj.tail(7).iterrows():
    id1 = row['catid_old']
    id2 = row['catid_new']
    geometry = row['geometry']
    
    clip_2tifs(id1, id2, geometry)

In [ ]:
df_reproj.tail(8)
df_reproj.to_file("example_df.gpkg", driver="GPKG")

In [ ]:
# idx = 2
# id1 = df_reproj['catid_old'][idx]
# id2 = df_reproj['catid_new'][idx]
# geometry = df_reproj['geometry'][idx]

# tif_file1 = glob.glob(f'{chm_dir}**/*{id1}*.tif')[0]
# tif_file2 = glob.glob(f'{chm_dir}**/*{id2}*.tif')[0]

# print(tif_file1)
# with rio.open(tif_file1) as src:
#     print(src.meta)
#     print(src.transform)
#     print(src.crs)

# print(tif_file2)
# with rio.open(tif_file2) as src:
#     print(src.meta)
#     print(src.transform)
#     print(src.crs)

In [ ]:
df_reproj.tail(5)

## Bad .tifs
#10300100D8887A00
#104001007A645900
#104001007B744C00
#10300100D644EA00

# One of these
# 10100100045A0600
# 10300100EB44FA00

still problem: 
gdalinfo WV02_20220714_M1BS_10300100D644EA00_sr_02m/WV02_20220714_M1BS_10300100D644EA00_sr_02m.cnn-chm-v1.tif


problem
WV02_20220801_M1BS_10300100D8887A00_sr_02m:
WV02_20220801_M1BS_10300100D8887A00_sr_02m.cnn-chm-v1.tif

WV02_20220802_M1BS_10300100D82F3A00_sr_02m:
WV02_20220802_M1BS_10300100D82F3A00_sr_02m.cnn-chm-v1.tif  WV02_20220802_M1BS_10300100D82F3A00_sr_02m.cnn-chm-v1.tif.aux.xml

WV02_20230805_M1BS_10300100EB44FA00_sr_02m:
WV02_20230805_M1BS_10300100EB44FA00_sr_02m.cnn-chm-v1.tif  WV02_20230805_M1BS_10300100EB44FA00_sr_02m.cnn-chm-v1.tif.aux.xml

WV03_20210718_M1BS_104001006C895900_sr_02m:
WV03_20210718_M1BS_104001006C895900_sr_02m.cnn-chm-v1.tif

WV03_20220728_M1BS_1040010079A0BD00_sr_02m:
WV03_20220728_M1BS_1040010079A0BD00_sr_02m.cnn-chm-v1.tif  WV03_20220728_M1BS_1040010079A0BD00_sr_02m.cnn-chm-v1.tif.aux.xml

problem
WV03_20220802_M1BS_104001007A645900_sr_02m:
WV03_20220802_M1BS_104001007A645900_sr_02m.cnn-chm-v1.tif.lock

problem
WV03_20220802_M1BS_104001007B744C00_sr_02m:
WV03_20220802_M1BS_104001007B744C00_sr_02m.cnn-chm-v1.tif.lock

WV03_20230716_M1BS_1040010086864D00_sr_02m:
WV03_20230716_M1BS_1040010086864D00_sr_02m.cnn-chm-v1.tif

WV03_20230716_M1BS_104001008A3AD000_sr_02m:
WV03_20230716_M1BS_104001008A3AD000_sr_02m.cnn-chm-v1.tif
